In [46]:
import pandas as pd
import ast
from PIL import Image
import numpy as np
import os
from collections import defaultdict

from analyze_masks import analyze_masks_and_list_exceptions

In [47]:
def categorize_data(data, column, num_categories):
    """ Categorize the data based on quantiles for a specific column """
    quantiles = data[column].quantile(np.linspace(0, 1, num_categories + 1)).drop_duplicates().sort_values()
    category_labels = [f"{int(100 * np.linspace(0, 1, len(quantiles))[i])}-{int(100 * np.linspace(0, 1, len(quantiles))[i+1])} percentile"
                       for i in range(len(quantiles)-1)]
    data[f'{column}_category'] = pd.cut(data[column], bins=quantiles, labels=category_labels, include_lowest=True)
    return data

In [48]:
def load_and_process_data(file_path, base_dir, masks_dir):
    data = pd.read_csv(file_path)
    confidence_columns = [col for col in data.columns if 'confidence' in col]
    for column in confidence_columns:
        data[column] = data[column].apply(ast.literal_eval)

    data['width'] = 0
    data['height'] = 0
    data['object_percentage'] = 0.0

    class_to_grayscale_map = analyze_masks_and_list_exceptions(masks_dir)

    for index, row in data.iterrows():
        class_id = row['id'].split('_')[0]
        picture_name = row['picture_name']
        picture_base_name = os.path.splitext(picture_name)[0]

        img_path = os.path.join(base_dir, 'train', class_id, picture_name)
        try:
            with Image.open(img_path) as img:
                width, height = img.size
                data.at[index, 'width'] = width
                data.at[index, 'height'] = height
        except FileNotFoundError:
            print(f"Image not found: {img_path}")

        mask_name = picture_base_name + '.png'
        mask_path = os.path.join(masks_dir, class_id, mask_name)
        try:
            with Image.open(mask_path) as mask:
                mask_array = np.array(mask)
                if class_id in class_to_grayscale_map:
                    relevant_value = class_to_grayscale_map[class_id]
                    object_pixels = np.sum(mask_array == relevant_value)
                    total_pixels = width * height
                    data.at[index, 'object_percentage'] = (object_pixels / total_pixels) * 100
        except FileNotFoundError:
            print(f"Mask not found for image: {mask_path}")

    data = categorize_data(data, 'object_percentage', 4)

    return data

In [49]:
def parse_synset_mapping(filepath):
    with open(filepath, 'r') as file:
        class_dict = {}
        for row_number, line in enumerate(file, start=1):
            class_id, description = line.strip().split(' ', 1)
            class_dict[class_id] = {
                "description": description,
                "value": row_number - 1
            }
    return class_dict

In [50]:
base_dir = '../data'
masks_dir = '../data/masks'
file_path = '../image_confidence_scores.csv'
synset_path = '../data/LOC_synset_mapping.txt'

processed_data = load_and_process_data(file_path, base_dir, masks_dir)
pd.DataFrame(processed_data).to_csv("test.csv", index=False)
class_dict = parse_synset_mapping(synset_path)

Class: n02412080, Most Common Nonzero Grayscale Value (by presence): 17, Presence Count: 982
    Images without the most common grayscale value (17): ['n02412080_13145.png', 'n02412080_1976.png', 'n02412080_6399.png', 'n02412080_16811.png', 'n02412080_2188.png', 'n02412080_19324.png', 'n02412080_18733.png', 'n02412080_2270.png', 'n02412080_16830.png', 'n02412080_16254.png', 'n02412080_26458.png', 'n02412080_3944.png', 'n02412080_1040.png', 'n02412080_10804.png', 'n02412080_13818.png', 'n02412080_11852.png', 'n02412080_791.png', 'n02412080_17063.png']
Class: n02107574, Most Common Nonzero Grayscale Value (by presence): 12, Presence Count: 997
    Images without the most common grayscale value (12): ['n02107574_3660.png', 'n02107574_142.png', 'n02107574_690.png']
Class: n01833805, Most Common Nonzero Grayscale Value (by presence): 3, Presence Count: 987
    Images without the most common grayscale value (3): ['n01833805_4117.png', 'n01833805_166.png', 'n01833805_8510.png', 'n01833805_885

In [51]:
column_names = processed_data.columns.tolist()
print(column_names)

['id', 'picture_name', 'original_confidence', 'desert_confidence', 'low_contrast_confidence', 'city_confidence', 'sky_confidence', 'jungle_confidence', 'no_bg_confidence', 'high_contrast_confidence', 'no_foreground_confidence', 'water_confidence', 'snow_confidence', 'indoor_confidence', 'mountain_confidence', 'width', 'height', 'object_percentage', 'object_percentage_category']


In [52]:
def extract_class_id_from_filename(filename):
    """Extracts the class ID from the filename."""
    return filename.split('_')[0]

In [53]:
def calculate_accuracy(data, confidence_column, class_dict):
    correct_predictions = 0
    total_predictions = 0

    for index, row in data.iterrows():
        true_class_id = row['id'].split('_')[0]
        
        if true_class_id in class_dict:
            true_class_value = class_dict[true_class_id]['value']
            
            predicted_confidences = row[confidence_column]
            
            if predicted_confidences:
                predicted_class_id = list(predicted_confidences[0].keys())[0]
                predicted_confidence = predicted_confidences[0][predicted_class_id]
                
                if true_class_value == predicted_class_id:
                    correct_predictions += 1
            
            total_predictions += 1
            

    return correct_predictions / total_predictions if total_predictions else 0

In [54]:
def calculate_and_print_all_accuracies(data, class_dict):
    modification_types = [
        'original', 'desert', 'low_contrast', 'city', 'sky', 'jungle',
        'no_bg', 'high_contrast', 'no_foreground', 'water', 'snow', 'indoor', 'mountain'
    ]
    accuracies = {}

    for mod_type in modification_types:
        column_name = f"{mod_type}_confidence"
        if column_name in data.columns:
            accuracy = calculate_accuracy(data, column_name, class_dict)
            accuracies[mod_type] = accuracy

    for mod_type, accuracy in accuracies.items():
        print(f"Accuracy for {mod_type} images: {accuracy:.2f}%")

In [55]:
calculate_and_print_all_accuracies(processed_data, class_dict)

Accuracy for original images: 0.89%
Accuracy for desert images: 0.76%
Accuracy for low_contrast images: 0.78%
Accuracy for city images: 0.76%
Accuracy for sky images: 0.76%
Accuracy for jungle images: 0.77%
Accuracy for no_bg images: 0.76%
Accuracy for high_contrast images: 0.78%
Accuracy for no_foreground images: 0.19%
Accuracy for water images: 0.72%
Accuracy for snow images: 0.75%
Accuracy for indoor images: 0.70%
Accuracy for mountain images: 0.70%


In [56]:
def calculate_per_class_accuracy(data, confidence_column, class_dict):
    class_correct = defaultdict(int)
    class_total = defaultdict(int)

    for index, row in data.iterrows():
        true_class_id = row['id'].split('_')[0]  
        
        if true_class_id in class_dict:
            true_class_value = class_dict[true_class_id]['value'] 

            predicted_confidences = row[confidence_column]
            if predicted_confidences:
                predicted_class_id = list(predicted_confidences[0].keys())[0] 
                predicted_confidence = predicted_confidences[0][predicted_class_id]

                class_total[true_class_value] += 1

                if true_class_value == predicted_class_id:
                    class_correct[true_class_value] += 1

    class_accuracies = {}
    for cls in class_total:
        if class_total[cls] > 0:
            class_accuracies[cls] = (class_correct[cls] / class_total[cls]) * 100

    return class_accuracies

In [57]:
def calculate_and_print_accuracies(data, class_dict):
    modification_types = [
        'original', 'desert', 'low_contrast', 'city', 'sky', 'jungle',
        'no_bg', 'high_contrast', 'no_foreground', 'water', 'snow', 'indoor', 'mountain'
    ]
    accuracies = {}
    
    for mod_type in modification_types:
        column_name = f"{mod_type}_confidence"
        if column_name in data.columns:
            accuracies[mod_type] = calculate_per_class_accuracy(data, column_name, class_dict)
    
    for mod_type, accuracy_dict in accuracies.items():
        print(f"Accuracy for {mod_type} images:")
        for cls, acc in accuracy_dict.items():
            class_id = [k for k, v in class_dict.items() if v['value'] == cls][0]
            description = class_dict[class_id]['description']
            print(f"  Class {cls} ({description}, {class_id}): {acc:.2f}%")
        print()  

In [58]:
calculate_and_print_accuracies(processed_data, class_dict)

Accuracy for original images:
  Class 348 (ram, tup, n02412080): 78.10%
  Class 238 (Greater Swiss Mountain dog, n02107574): 74.60%
  Class 94 (hummingbird, n01833805): 96.30%
  Class 285 (Egyptian cat, n02124075): 78.20%
  Class 349 (bighorn, bighorn sheep, cimarron, Rocky Mountain bighorn, Rocky Mountain sheep, Ovis canadensis, n02415577): 85.70%
  Class 229 (Old English sheepdog, bobtail, n02105641): 94.10%
  Class 283 (Persian cat, n02123394): 96.40%
  Class 13 (junco, snowbird, n01534433): 97.47%
  Class 235 (German shepherd, German shepherd dog, German police dog, alsatian, n02106662): 87.40%
  Class 15 (robin, American robin, Turdus migratorius, n01558993): 98.10%

Accuracy for desert images:
  Class 348 (ram, tup, n02412080): 66.20%
  Class 238 (Greater Swiss Mountain dog, n02107574): 75.10%
  Class 94 (hummingbird, n01833805): 67.70%
  Class 285 (Egyptian cat, n02124075): 82.50%
  Class 349 (bighorn, bighorn sheep, cimarron, Rocky Mountain bighorn, Rocky Mountain sheep, Ovis c

In [59]:
def calculate_dynamic_accuracy_by_category(data, confidence_columns, class_dict):
    percentage_categories = data['object_percentage_category'].unique()
    
    results = {
        category: {
            modification: defaultdict(lambda: {'total': 0, 'correct': 0})
            for modification in confidence_columns
        } for category in percentage_categories
    }
    
    for index, row in data.iterrows():
        true_class_id = row['id'].split('_')[0]
        
        if true_class_id in class_dict:
            true_class_value = class_dict[true_class_id]['value']
            current_category = row['object_percentage_category']
            
            for column in confidence_columns:
                predicted_confidences = row[column]
                
                if predicted_confidences:
                    predicted_class_id = list(predicted_confidences[0].keys())[0]
                    
                    results[current_category][column][true_class_value]['total'] += 1
                    
                    if true_class_value == predicted_class_id:
                        results[current_category][column][true_class_value]['correct'] += 1

    accuracies = {category: {modification: {} for modification in confidence_columns} for category in percentage_categories}
    for category in results:
        for modification in results[category]:
            for cls in results[category][modification]:
                total = results[category][modification][cls]['total']
                correct = results[category][modification][cls]['correct']
                if total > 0:
                    accuracies[category][modification][cls] = (correct / total) * 100

    return accuracies

In [60]:
def calculate_and_print_dynamic_accuracies(data, class_dict):
    modification_types = [
        'original', 'desert', 'low_contrast', 'city', 'sky', 'jungle',
        'no_bg', 'high_contrast', 'no_foreground', 'water', 'snow', 'indoor', 'mountain'
    ]

    confidence_columns = [f"{mod}_confidence" for mod in modification_types if f"{mod}_confidence" in data.columns]

    dynamic_accuracies = calculate_dynamic_accuracy_by_category(data, confidence_columns, class_dict)

    for category in dynamic_accuracies:
        print(f"\nCategory: {category}")
        for modification in dynamic_accuracies[category]:
            print(f"  Accuracy for {modification.replace('_confidence', '')} images:")
            for cls, acc in dynamic_accuracies[category][modification].items():
                class_id = [k for k, v in class_dict.items() if v['value'] == cls][0]
                description = class_dict[class_id]['description']
                print(f"    Class {cls} ({description}, {class_id}): {acc:.2f}%")
        print()

In [61]:
calculate_and_print_dynamic_accuracies(processed_data, class_dict)


Category: 0-25 percentile
  Accuracy for original images:
    Class 348 (ram, tup, n02412080): 57.39%
    Class 238 (Greater Swiss Mountain dog, n02107574): 62.96%
    Class 94 (hummingbird, n01833805): 94.28%
    Class 285 (Egyptian cat, n02124075): 60.38%
    Class 349 (bighorn, bighorn sheep, cimarron, Rocky Mountain bighorn, Rocky Mountain sheep, Ovis canadensis, n02415577): 88.69%
    Class 229 (Old English sheepdog, bobtail, n02105641): 91.10%
    Class 283 (Persian cat, n02123394): 77.19%
    Class 13 (junco, snowbird, n01534433): 95.68%
    Class 235 (German shepherd, German shepherd dog, German police dog, alsatian, n02106662): 80.71%
    Class 15 (robin, American robin, Turdus migratorius, n01558993): 96.61%
  Accuracy for desert images:
    Class 348 (ram, tup, n02412080): 29.55%
    Class 238 (Greater Swiss Mountain dog, n02107574): 49.38%
    Class 94 (hummingbird, n01833805): 48.34%
    Class 285 (Egyptian cat, n02124075): 39.62%
    Class 349 (bighorn, bighorn sheep, ci